In [ ]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from datetime import date

sys.path.append('../')

from util.chart_methods import draw_investment_log
from util.math_methods import round_down

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
portfolio_name = 'test'

In [ ]:
total_policies = 6
df_list = []
for policy_id in range(0, total_policies):
    df = pd.read_csv('../../logs/{0}-{1}.log'.format(portfolio_name, policy_id), parse_dates=['date'], date_parser=pd.to_datetime)
    df_list.append(df)

In [ ]:
for policy_id in range(0, total_policies):
    draw_investment_log(df_list[policy_id], xlabel='Trade Date', ylabel='Asset (unit: 10K)', title='Assets Changes Table of Policy ' + str(policy_id))

In [ ]:
def print_cagr(df):
    start_date = date.fromtimestamp(df.date.values[0].astype(int) / 1e9)
    end_date = date.fromtimestamp(df.date.values[-1].astype(int) / 1e9)
    years = end_date.year - start_date.year if end_date.year > start_date.year else 1
    initial_total = df.total.values[0]
    total = df.total.values[-1]
    cagr = round_down((total / initial_total) ** (1 / years) - 1)
    return_rate = round_down(total / initial_total * 100)
    print('Initial investment is {0}, after {1} years, now is {2}, return rate is {3}%, CAGR: {4}'.format(initial_total, years, total, return_rate, cagr))

for policy_id in range(0, total_policies):
    print_cagr(df_list[policy_id])

In [ ]:
df_trade_list = []
for policy_id in range(0, total_policies):
    df_trade = pd.read_csv('../../logs/trade_{0}-{1}.log'.format(portfolio_name, policy_id), parse_dates=['date','hold_date'], date_parser=pd.to_datetime)
    df_trade_list.append(df_trade)

In [ ]:
def get_win_loss(policy_id):
    df_trade = df_trade_list[policy_id]
    return df_trade[['ts_code','status']].groupby(['status']).count().rename(columns={"ts_code": "policy_{0}".format(policy_id)})

df_win_loss = get_win_loss(0)
for policy_id in range(1, total_policies):
    df_win_loss = df_win_loss.join(get_win_loss(policy_id))

df_win_loss.T

In [ ]:
def get_stock_benefit(policy_id):
    df_trade = df_trade_list[policy_id]
    df_benefit = df_trade[['ts_code', 'benefit']].groupby(['ts_code']).sum(['benefit']).rename(columns={"benefit": "sum_{0}".format(policy_id)})
    df_win = df_trade[df_trade['status']=='win']
    df_win_sum = df_win[['ts_code', 'benefit']].groupby(['ts_code']).sum(['benefit']).rename(columns={"benefit": "win_{0}".format(policy_id)})
    df_win_count = df_win[['ts_code', 'benefit']].groupby(['ts_code']).count().rename(columns={"benefit": "w_cnt_{0}".format(policy_id)})
    df_loss = df_trade[df_trade['status']=='loss']
    df_loss_sum = df_loss[['ts_code', 'benefit']].groupby(['ts_code']).sum(['benefit']).rename(columns={"benefit": "loss_{0}".format(policy_id)})
    df_loss_count = df_loss[['ts_code', 'benefit']].groupby(['ts_code']).count().rename(columns={"benefit": "l_cnt_{0}".format(policy_id)})
    df_benefit = df_benefit.join(df_win_sum).join(df_loss_sum).join(df_win_count).join(df_loss_count)
    return df_benefit

In [ ]:
df_benefit = get_stock_benefit(0)
for policy_id in range(1, total_policies):
    df_benefit = pd.merge(df_benefit, get_stock_benefit(policy_id), on='ts_code', how='outer')

policy_id=4
df_benefit.sort_values(by=['sum_{0}'.format(policy_id)], ascending=False)

In [ ]:
df_trade_list[policy_id].tail(60)

In [ ]:
policy_id=5
df_benefit.sort_values(by=['sum_{0}'.format(policy_id)], ascending=False)

In [ ]:
df_trade_list[policy_id].tail(60)